In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
data = pd.read_csv("A:\Download\manhattan.csv")

In [3]:
data.head()

,GRID_ID,idle_count,relief_stands,geometry,dropoff_co,parks,rest_cheap,violations
0,AS-54,0.0,0,POLYGON ((-74.04589966952779 40.69033272469935...,0.0,0.0,0,0.0
1,AS-55,0.0,0,POLYGON ((-74.04589966952779 40.69033272469935...,0.0,0.0,0,0.0
2,AT-53,0.0,0,POLYGON ((-74.04109861205649 40.70049294329179...,0.5,0.0,0,0.0
3,AT-54,0.0,0,POLYGON ((-74.04146718007551 40.69659112724123...,0.0,0.0,0,0.0
4,AT-55,0.0,0,POLYGON ((-74.04388559622677 40.69018482394773...,0.0,0.0,0,0.0


In [4]:
subdat = data[["GRID_ID", "idle_count", "dropoff_co", "parks", "rest_cheap", "violations"]]

In [36]:
dat = subdat[["idle_count", "dropoff_co", "parks", "rest_cheap", "violations"]]

In [37]:
dat = (dat - dat.min()) / (dat.max() - dat.min())

In [42]:
dat["GRID_ID"] = subdat["GRID_ID"]

In [93]:
scores_m = np.zeros([dat.shape[0],50])
sort_list = ["violations", "idle_count", "dropoff_co", "rest_cheap", "parks"]
np.random.seed(100)
for _ in range(50):
    weights = np.array(sorted(np.random.dirichlet(np.ones(5)))[::-1])
    vals = dat[sort_list].values
    scores = vals.dot(weights)
    scores_m[:,_] = vals.dot(weights).reshape(vals.shape[0])
    print "score " + str(_) + " is:"
    for i,v in enumerate(sort_list):
        print v + " is: " + str(weights[i])
    print "end \n"

score 0 is:
violations is: 0.527675618273
idle_count is: 0.222061638472
dropoff_co is: 0.156512657033
rest_cheap is: 0.0924102738331
parks is: 0.00133981238829
end 

score 1 is:
violations is: 0.437916683527
idle_count is: 0.278339616274
dropoff_co is: 0.214438173667
rest_cheap is: 0.0368303281448
parks is: 0.0324751983858
end 

score 2 is:
violations is: 0.734443291996
idle_count is: 0.0820937320416
dropoff_co is: 0.0776723327425
rest_cheap is: 0.0678296663018
parks is: 0.0379609769185
end 

score 3 is:
violations is: 0.498239701288
idle_count is: 0.219488758936
dropoff_co is: 0.216325766619
rest_cheap is: 0.0415006027559
parks is: 0.0244451704018
end 

score 4 is:
violations is: 0.495113750549
idle_count is: 0.299439957107
dropoff_co is: 0.0994330051621
rest_cheap is: 0.0720774353215
parks is: 0.0339358518606
end 

score 5 is:
violations is: 0.671019186478
idle_count is: 0.197141934401
dropoff_co is: 0.122932447763
rest_cheap is: 0.00649582781212
parks is: 0.00241060354552
end 

scor

In [96]:
for _ in range(10):
    data["score_"+ str(_)] = scores_m[:,_]

In [99]:
data.to_csv("withscores.csv")

In [14]:
import geopandas as gpd
from fiona.crs import from_epsg

In [130]:
dat = gpd.GeoDataFrame.from_csv("withscores.csv")

In [57]:
dat.head()

,GRID_ID,idle_count,relief_stands,geometry,dropoff_co,parks,rest_cheap,violations,score_0,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,ave_score
158,BG-42,308.202740,2,POLYGON ((-73.97944485982342 40.74039365532082...,2768.104110,0.0,50,2011.0,0.904409,0.838391,0.915265,0.845284,0.906186,0.923560,0.867573,0.850255,0.843175,0.840874,0.873497
93,BD-38,264.106849,2,POLYGON ((-73.99369078885718 40.75916533252825...,1898.265753,0.0,50,1190.0,0.637647,0.592980,0.593970,0.583431,0.648782,0.606040,0.637160,0.612814,0.611784,0.613548,0.613816
137,BF-42,127.704110,4,POLYGON ((-73.98419360103942 40.74247996463384...,4357.375342,0.0,50,1065.0,0.561880,0.518356,0.539435,0.531770,0.520621,0.520525,0.518979,0.565115,0.549580,0.511355,0.533762
115,BE-39,38.482192,1,POLYGON ((-73.98894125338362 40.75290830780834...,5089.923288,0.0,50,862.0,0.460805,0.416149,0.449708,0.440708,0.394425,0.408660,0.394529,0.478347,0.453845,0.399764,0.429694
97,BD-42,135.002740,1,POLYGON ((-73.99369236883631 40.74248087656221...,2531.545205,0.0,50,738.0,0.440268,0.397474,0.401574,0.399189,0.421114,0.383826,0.415615,0.439202,0.427208,0.410581,0.413605


In [131]:
dat["ave_score"] = dat[["score_" + str(i) for i in range(10)]].mean(axis=1)

In [132]:
dat[["parks", "dropoff_co","GRID_ID"]][dat["parks"] != 0].sort_values(["dropoff_co"], ascending=False).head(20).to_csv("parks.csv")

In [133]:
dat.sort_values("ave_score", ascending=False).to_csv("withscores.csv")

In [65]:
request_dat = gpd.GeoDataFrame.from_file("stand_requests\stand_requests.shp")

In [71]:
request_dat = request_dat.set_geometry("geometry")
request_dat.crs = from_epsg(4326)

In [72]:
haxgon =  gpd.GeoDataFrame.from_file("Hexagon_2km2/Hexagon_clipped/hexagons_clipped.shp")

In [108]:
haxgon = haxgon.set_geometry("geometry")
haxgon.crs = from_epsg(2263)
haxgon = haxgon.to_crs(epsg=4326)

In [117]:
request_dat["GRID_ID"] = request_dat.apply(lambda x: haxgon[haxgon.geometry.contains(x.geometry)]["GRID_ID"].values[0], axis=1)

In [120]:
manhattan = request_dat[request_dat["borough"] == "Manhattan"]

In [134]:
dat_sort = dat.sort_values("idle_count", ascending=False)

In [259]:
features = ["parks", "rest_cheap"]

In [198]:
request_ = dat_sort.merge(request_dat, on="GRID_ID")[features + ["GRID_ID"]]

In [195]:
dat_ = dat_sort[features + ["GRID_ID", "relief_stands"]]

In [199]:
request_["flag"] = 1

In [210]:
flags1 = dat_[dat["relief_stands"] != 0][features + ["GRID_ID"]]

B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [214]:
flags1["flag"] = np.append(np.ones(10), np.zeros(flags1.shape[0]-10))

In [233]:
final = flags1.append(request_).drop_duplicates()

In [237]:
from sklearn import linear_model 

In [260]:
y_train = final["flag"]
x_train = final[features]

In [263]:
for c in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]:
    lgc = linear_model.LogisticRegression(C=c)
    lgc.fit(x_train, y_train)
    print ("c is: " + str(c) + " score is: " + str(lgc.score(x_train, y_train)))

c is: 1e-05 score is: 0.536585365854
c is: 0.0001 score is: 0.536585365854
c is: 0.001 score is: 0.536585365854
c is: 0.01 score is: 0.536585365854
c is: 0.1 score is: 0.536585365854
c is: 1 score is: 0.609756097561
c is: 10.0 score is: 0.634146341463
c is: 100.0 score is: 0.634146341463
c is: 1000.0 score is: 0.634146341463
c is: 10000.0 score is: 0.634146341463
c is: 100000.0 score is: 0.634146341463
c is: 1000000.0 score is: 0.634146341463


In [264]:
lgc = linear_model.LogisticRegression(C=10)
lgc.fit(x_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [265]:
prob = lgc.predict_proba(dat_[dat["relief_stands"] == 0][features])[:,1]

B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [268]:
f_dat = dat_[dat["relief_stands"] == 0]
f_dat["prob"] = prob

B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
B:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [1]:
import pandas as pd
import numpy as np

In [3]:
f_dat = pd.read_csv("logsc.csv")

In [4]:
f_dat.reset_index(drop=True, inplace=True)

In [5]:
import pulp

In [6]:
trs_model = pulp.LpProblem("trs Problem", pulp.LpMaximize)

In [7]:
x = pulp.LpVariable.dict('x_%s', range(f_dat.shape[0]))
y = pulp.LpVariable.dict('y_%s', range(f_dat.shape[0]), lowBound=0, upBound=1, cat="Binary")

In [8]:
trs_model += sum([f_dat.loc[i, "violations"] * x[i] for i in range(f_dat.shape[0])])

In [9]:
trs_model += sum([y[i] for i in range(f_dat.shape[0])]) <= 20
for i in range(f_dat.shape[0]):
    zero = np.zeros(f_dat.shape[0])
    zero[i] = 1
    trs_model += sum(x[i]*zero[i] for i in range(f_dat.shape[0])) <= f_dat.loc[i, "idle_count"]
    trs_model += sum(x[i]*zero[i] for i in range(f_dat.shape[0])) <= f_dat.loc[i, "prob"] * y[i]

In [10]:
trs_model.solve()

1

In [11]:
ids = []
for v in trs_model.variables():
    if v.name.startswith("y") & (v.varValue == 1):
        ids.append(f_dat.loc[int(v.name[2:]), "GRID_ID"])

In [12]:
ids

['BE-41',
 'BB-40',
 'BM-26',
 'BC-37',
 'BP-27',
 'BG-45',
 'BE-40',
 'BI-39',
 'BH-40',
 'BI-38',
 'BH-39',
 'BM-27',
 'BF-33',
 'BH-41',
 'BD-37',
 'BG-33',
 'BK-28',
 'BF-37',
 'BG-41',
 'BN-34']